In [ ]:
from typing import *
from IPython.display import HTML, Javascript, display
import ipywidgets as wi
from PIL import Image
from pathlib import Path
from random import Random
import pickle
PX = 128
root = Path("/data/natsuki")
def bucket(_id: str) -> str:
    return _id[-3:].zfill(4)

In [ ]:
class Pannel(wi.VBox):
    def fname2id(self, fname: str) -> str:
        return fname.split("/")[-1].split(".")[0]
    def on_click(self, event) -> None:
        if not event["new"]:
            self.bag.add(self.fname)
        else:
            self.bag.discard(self.fname)
        self.output.clear_output()
        with self.output:
            print(self.id, len(self.bag))
    def __init__(
        self,
        fname: str,
        bag: Set[str],
        output: wi.Output,
    ) -> None:
        self.fname = fname
        self.bag = bag
        self.output = output
        self.id = self.fname2id(fname)
        with open(fname, "rb") as f:
            buf = f.read()
        self.image = wi.Image(value=buf, format='png', width=PX, height=PX)
        self.button = wi.ToggleButton(description=self.id, layout=wi.Layout(width=f"{PX}px"), value=self.fname not in bag, button_style="info")
        self.button.observe(self.on_click, "value")
        super().__init__([self.image, self.button])

In [ ]:
class Viewer(wi.Box):
    def __len__(self) -> int:
        return (len(self.fnames)-1)//self.N+1
    def __getitem__(self, i: int) -> List[Pannel]:
        assert i in range(len(self))
        return [
            Pannel(fname, self.bag, self.output)
            for fname in self.fnames[i*self.N:(i+1)*self.N]
        ]
    def refresh(self) -> None:
        self.status.clear_output()
        with self.status:
            print(f"{self.i}/{len(self)}", len(self.fnames))
        self.children = tuple(self[self.i]+[self.controller])
    def on_click_p(self, event) -> None:
        self.i -= 1; self.i %= len(self); self.refresh()
    def on_click_n(self, event) -> None:
        self.i += 1; self.i %= len(self); self.refresh()
    def save(self, event=None) -> None:
        self.status.clear_output()
        with self.status:
            print("saving...")
        (root/"viewer").mkdir(exist_ok=True)
        with open(root/"viewer"/f"{self.hash}.pkl", "wb") as f:
            pickle.dump(self.bag, f, protocol=4)
        self.status.clear_output()
        with self.status:
            print("saved!")
    def load(self, event=None) -> None:
        self.status.clear_output()
        with self.status:
            print("loading...")
        with open(root/"viewer"/f"{self.hash}.pkl", "rb") as f:
            self.bag = pickle.load(f)
        self.refresh()
    def __init__(
        self,
        fnames: Set[str],
        N: int = 23,
        seed: int = 0,
    ) -> None:
        self.fnames = list(set(fnames))
        self.N = N
        self.seed = seed
        self.fnames.sort()
        Random(self.seed).shuffle(self.fnames)
        self.fnames = tuple(self.fnames)
        self.hash = hash(self.fnames)
        self.i = 0
        self.bag = set()
        self.button_n = wi.Button(description="next", layout=wi.Layout(width=f"{PX}px"))
        self.button_n.on_click(self.on_click_n)
        self.button_p = wi.Button(description="prev", layout=wi.Layout(width=f"{PX}px"))
        self.button_p.on_click(self.on_click_p)
        self.button_s = wi.Button(description="save", layout=wi.Layout(width=f"{PX}px"))
        self.button_s.on_click(self.save)
        self.button_l = wi.Button(description="load", layout=wi.Layout(width=f"{PX}px"))
        self.button_l.on_click(self.load)
        self.status = wi.Output(layout=wi.Layout(width=f"{PX}px"))
        self.output = wi.Output(layout=wi.Layout(width=f"{PX}px"))
        self.controller = wi.VBox([self.button_n, self.button_p, self.button_s, self.button_l, self.status, self.output])
        self.layout = layout = wi.Layout(flex_flow="row wrap")
        super().__init__()
        self.refresh()

In [ ]:
wi.VBox([
    wi.HBox([wi.ToggleButton(description=k, button_style=k, value=v) for v in range(2)])
    for k in  ['primary','warning','info','success','danger']
])